In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO


In [5]:
# 전날
q = '''
select uid
from service1_quicket.product_info
where create_date >= '2021-06-15 00:00:00' and create_date <= '2021-06-15 23:59:59'
group by 1
'''
upload_yesterday = pd.read_sql(q, con=bun_dw)


In [9]:
uid_list = upload_yesterday.uid.astype(str).to_list()

In [7]:
seller = pd.read_excel('powerSeller.xlsx', sheet_name='result', engine='openpyxl').uid.astype(str).to_list()
final_uid_list = []
for uid in uid_list:
    if uid not in seller:
        final_uid_list.append(uid)

In [37]:
len(uid_list)

34208

In [39]:
len(seller)

174929

In [44]:
len(final_uid_list)

31753

In [10]:
uids = ','.join('\'' + str(u) + '\'' for u in uid_list)

In [45]:
f_uids = ','.join('\'' + str(u) + '\'' for u in final_uid_list)

#### 파워셀러 점수 스코어링 피쳐들
- 라이브중인 상품 수
- 상품 등록 수
- 상품 등록 시간
- 업 사용 수

In [11]:
# 2020/06/09 기준
q = f'''
SELECT UID,
       count(CASE
                 WHEN status = 0 THEN 1
             END) AS live_cnt,
       count(DISTINCT id) AS p_cnt
FROM service1_quicket.product_info
WHERE UID IN ({uids})
GROUP BY 1
'''
products = pd.read_sql(q, con=bun_dw)

In [12]:
q = f'''
SELECT UID,
       min(extract(hour
                   FROM create_date)) AS earliest,
       max(extract(hour
                   FROM create_date)) AS latest,
       latest - earliest AS diff
FROM service1_quicket.product_info
WHERE UID IN ({uids})
  AND create_date BETWEEN '2021-06-13 00:00:00' AND '2021-06-15 23:59:59'
GROUP BY 1
'''
upload = pd.read_sql(q, con=bun_dw)

In [13]:
q = f'''
SELECT a.uid,
       a.top_category,
       a.p_cnt_in_top_category,
       b.distinct_category_cnt
FROM
  (SELECT UID,
          category AS top_category,
          p_cnt_in_category AS p_cnt_in_top_category
   FROM
     (SELECT UID,
             left(category_id, 3) AS category,
             count(DISTINCT id) AS p_cnt_in_category,
             row_number() OVER (PARTITION BY UID
                                ORDER BY count(DISTINCT id) DESC) AS ROW
      FROM service1_quicket.product_info
      WHERE UID IN ({uids}) AND status = 0
      GROUP BY 1,
               2)
   WHERE ROW = 1) a
JOIN
  (SELECT UID,
          count(DISTINCT left(category_id, 3)) AS distinct_category_cnt
   FROM service1_quicket.product_info
   WHERE UID IN ({uids})
   GROUP BY 1) b ON a.uid = b.uid
'''
category = pd.read_sql(q, con=bun_dw)

In [14]:
#일주일간의 up plus
q = f'''
SELECT UID,
       count(*) AS up_plus_purchase_cnt,
       sum(qty) AS up_plus_cnt,
       sum(pay_point) AS pay_point,
       sum(pay_free) AS pay_free,
       sum(pay_point) + sum(pay_free) AS total_point
FROM service1_quicket.ad_up_plus
WHERE UID IN ({uids})
  AND created_at >= '2021-06-09'
  AND created_at <= '2021-06-15'
GROUP BY 1'''
up_plus = pd.read_sql(q, con=bun_dw)

In [15]:
# 일주일간 SA
q = f'''
SELECT UID,
       count(DISTINCT id) AS ad_id_cnt,
       sum(total_budget) AS total_budget,
       sum(paid_budget) AS total_paid_budget,
       sum(free_budget) AS total_free_budget
FROM service1_quicket.ad_set
WHERE created_at >= '2021-06-09'
  AND created_at <= '2021-06-15'
  AND UID IN ({uids})
GROUP BY 1
'''
sa = pd.read_sql(q, con=bun_dw)

In [16]:
q = f'''
SELECT content_owner AS UID,
       count(*) AS total_click_cnt,
       count(DISTINCT content_id) AS p_cnt,
       cast(total_click_cnt AS float)/p_cnt AS avg_click_cnt
FROM bun_log_db.app_event_type_view
WHERE content_owner IN ({uids})
  AND YEAR||MONTH||DAY = '20210615'
GROUP BY 1
'''
click = pd.read_sql(q, con=bun_dw)

In [17]:
q = f'''
SELECT content_owner AS UID,
       count(*) AS total_imp_cnt,
       count(DISTINCT content_id) AS p_cnt,
       cast(total_imp_cnt AS float)/p_cnt AS avg_imp_cnt
FROM bun_log_db.app_event_type_impression
WHERE content_owner IN ({uids})
  AND YEAR||MONTH||DAY = '20210615'
GROUP BY 1
'''
imp = pd.read_sql(q, con=bun_dw)

In [18]:
q = f'''
SELECT id AS UID,
       favorite_count AS follower_cnt,
       review_count,
       grade
FROM service1_quicket.user_
WHERE id IN ({uids})
'''
user = pd.read_sql(q, con=bun_dw)

### 모든 테이블 JOIN

In [19]:
tables = [products, upload, category, up_plus, sa, click, imp, user]
for t in tables:
    t['uid'] = t['uid'].astype(str)

In [20]:
df = pd.merge(products, upload, on='uid', how='outer')
df = pd.merge(df, category , on='uid', how='outer')
df = pd.merge(df, up_plus, on='uid', how='outer')
df = pd.merge(df, sa, on='uid', how='outer')
df = pd.merge(df, user, on='uid', how='outer')

In [21]:
click.uid = click.uid.astype(str)
imp.uid = imp.uid.astype(str)

cm = pd.merge(click, imp, on='uid', how='outer')

In [22]:
df = pd.merge(df, cm, on='uid', how='outer')

In [23]:
df

,uid,live_cnt,p_cnt,earliest,latest,diff,top_category,p_cnt_in_top_category,distinct_category_cnt,up_plus_purchase_cnt,...,total_free_budget,follower_cnt,review_count,grade,total_click_cnt,p_cnt_x,avg_click_cnt,total_imp_cnt,p_cnt_y,avg_imp_cnt
0,74282013,8,1995,0,23,23,600,8.0,2.0,NaN,...,NaN,12,0,0,439.0,37.0,11.864865,17455.0,72.0,242.430556
1,1705879,41,3604,1,15,14,310,33.0,14.0,NaN,...,NaN,93,73,710,64.0,12.0,5.333333,1069.0,35.0,30.542857
2,1266155,482,1284,15,15,0,400,482.0,12.0,NaN,...,NaN,13812,2166,20789,1094.0,249.0,4.393574,42942.0,587.0,73.155026
3,4756388,51,405,14,21,7,310,22.0,11.0,NaN,...,NaN,5,7,54,193.0,35.0,5.514286,3646.0,50.0,72.920000
4,74514703,11,380,6,20,14,320,6.0,5.0,NaN,...,NaN,1,1,10,467.0,25.0,18.680000,10503.0,65.0,161.584615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33138,75907562,1,1,0,0,0,900,1.0,1.0,NaN,...,NaN,0,0,0,1.0,1.0,1.000000,304.0,1.0,304.000000
33139,3611880,1,1,9,9,0,750,1.0,1.0,NaN,...,NaN,0,0,0,81.0,1.0,81.000000,1924.0,1.0,1924.000000
33140,75077079,1,1,1,1,0,999,1.0,1.0,NaN,...,NaN,1,0,0,92.0,1.0,92.000000,488.0,1.0,488.000000
33141,76793069,1,1,19,19,0,910,1.0,1.0,NaN,...,NaN,0,0,0,16.0,1.0,16.000000,183.0,1.0,183.000000


In [24]:
df.columns

Index(['uid', 'live_cnt', 'p_cnt', 'earliest', 'latest', 'diff',
       'top_category', 'p_cnt_in_top_category', 'distinct_category_cnt',
       'up_plus_purchase_cnt', 'up_plus_cnt', 'pay_point', 'pay_free',
       'total_point', 'ad_id_cnt', 'total_budget', 'total_paid_budget',
       'total_free_budget', 'follower_cnt', 'review_count', 'grade',
       'total_click_cnt', 'p_cnt_x', 'avg_click_cnt', 'total_imp_cnt',
       'p_cnt_y', 'avg_imp_cnt'],
      dtype='object')

In [25]:
df = df.fillna(0)

## Remove uids 

In [26]:
exclude = pd.read_csv('exclude_uid.csv')
exclude.head()

,uid
0,76402316
1,75466464
2,13079006
3,12845403
4,76254793


In [28]:
len(exclude)

600

In [29]:
exclude['uid'] = exclude['uid'].astype(str)
df = df[~df['uid'].isin(exclude['uid'])]

### SCORING

In [30]:
from sklearn.preprocessing import MinMaxScaler
cols = ['uid', 'live_cnt', 'p_cnt', 
        'up_plus_purchase_cnt', 'up_plus_cnt', 'total_point', 
        'ad_id_cnt', 'total_budget', 
        'follower_cnt', 'review_count', 'grade', 
        'total_click_cnt', 'avg_click_cnt', 'total_imp_cnt', 'avg_imp_cnt']
s = df[cols]

In [31]:
import numpy as np
s['if_work_hr'] = np.where(((df['diff']>=8) & (df['diff'] <= 10)) | (df['diff'] == 23), 0.5, 0)
s['if_category'] = np.where((df['p_cnt_in_top_category'] >= 10) & (df['distinct_category_cnt'] <= 3), 0.5, 0)

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
cols = ['live_cnt', 'p_cnt', 
        'up_plus_purchase_cnt', 'up_plus_cnt', 'total_point', 
        'ad_id_cnt', 'total_budget', 
        'follower_cnt', 'review_count', 'grade', 
        'total_click_cnt', 'avg_click_cnt', 'total_imp_cnt', 'avg_imp_cnt']
s[cols] = MinMaxScaler().fit_transform(s[cols])

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [33]:
s

,uid,live_cnt,p_cnt,up_plus_purchase_cnt,up_plus_cnt,total_point,ad_id_cnt,total_budget,follower_cnt,review_count,grade,total_click_cnt,avg_click_cnt,total_imp_cnt,avg_imp_cnt,if_work_hr,if_category
0,74282013,0.000408,0.034751,0.0,0.0,0.0,0.0,0.0,0.000441,0.000000,0.000000,0.034283,0.021417,0.062791,0.029016,0.5,0.0
1,1705879,0.002093,0.062792,0.0,0.0,0.0,0.0,0.0,0.003419,0.005921,0.005956,0.004998,0.009627,0.003846,0.003656,0.0,0.0
2,1266155,0.024604,0.022360,0.0,0.0,0.0,0.0,0.0,0.507719,0.175698,0.174403,0.085435,0.007931,0.154475,0.008756,0.0,0.0
3,4756388,0.002603,0.007041,0.0,0.0,0.0,0.0,0.0,0.000184,0.000568,0.000453,0.015072,0.009954,0.013116,0.008728,0.0,0.0
4,74514703,0.000562,0.006605,0.0,0.0,0.0,0.0,0.0,0.000037,0.000081,0.000084,0.036470,0.033718,0.037782,0.019340,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33138,75907562,0.000051,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000078,0.001805,0.001094,0.036385,0.0,0.0
33139,3611880,0.000051,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.006326,0.146209,0.006921,0.230281,0.0,0.0
33140,75077079,0.000051,0.000000,0.0,0.0,0.0,0.0,0.0,0.000037,0.000000,0.000000,0.007185,0.166065,0.001755,0.058408,0.0,0.0
33141,76793069,0.000051,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001250,0.028881,0.000658,0.021903,0.0,0.0


In [34]:
s.columns

Index(['uid', 'live_cnt', 'p_cnt', 'up_plus_purchase_cnt', 'up_plus_cnt',
       'total_point', 'ad_id_cnt', 'total_budget', 'follower_cnt',
       'review_count', 'grade', 'total_click_cnt', 'avg_click_cnt',
       'total_imp_cnt', 'avg_imp_cnt', 'if_work_hr', 'if_category'],
      dtype='object')

In [35]:
def score(live_cnt, p_cnt, up_plus_purchase_cnt, up_plus_cnt, total_point, ad_id_cnt, total_budget, follower_cnt, review_count, grade, avg_click_cnt, avg_imp_cnt, if_work_hr, if_category):
    sc = s['live_cnt'] * live_cnt \
    + s['p_cnt'] * p_cnt \
    + s['up_plus_purchase_cnt'] * up_plus_purchase_cnt \
    + s['up_plus_cnt'] * up_plus_cnt \
    + s['total_point'] * total_point \
    + s['ad_id_cnt'] * ad_id_cnt \
    + s['total_budget'] * total_budget \
    + s['follower_cnt'] * follower_cnt \
    + s['review_count'] * review_count \
    + s['grade'] * grade \
    + s['avg_click_cnt'] * avg_click_cnt \
    + s['avg_imp_cnt'] * avg_imp_cnt \
    + s['if_work_hr'] * if_work_hr \
    + s['if_category'] * if_category
    return sc

In [36]:
s['score'] = score(5, 1, 2, 3, 3, 6, 3, 4, 4, 3, 5, 2, 5, 5)
features = ['live_cnt', 'p_cnt', 'up_plus_purchase_cnt', 'up_plus_cnt', 'total_point', 'ad_id_cnt', 'total_budget', 'follower_cnt', 'review_count', 'grade', 'avg_click_cnt', 'avg_imp_cnt', 'if_work_hr', 'if_category']
s.sort_values('score', ascending=False)[['uid', 'score'] + features][:30]

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,uid,score,live_cnt,p_cnt,up_plus_purchase_cnt,up_plus_cnt,total_point,ad_id_cnt,total_budget,follower_cnt,review_count,grade,avg_click_cnt,avg_imp_cnt,if_work_hr,if_category
378,3947717,19.126394,0.022307,0.088254,1.000000,1.000000,1.000000,1.000000,0.925004,0.209124,0.147388,0.143489,0.046884,0.030330,0.0,0.0
25192,592959,12.309743,0.006687,0.049233,0.083333,0.188679,0.183150,0.013514,0.041095,0.909499,1.000000,1.000000,0.016454,0.010144,0.0,0.0
24721,2624438,11.441002,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350316,0.216175,0.216676,0.004447,0.001387,0.5,0.0
1229,6972746,11.094439,0.004645,0.058592,0.000000,0.000000,0.000000,0.722973,1.000000,0.133877,0.059458,0.059597,0.034560,0.024925,0.0,0.5
5727,4879259,8.534829,0.000766,0.008958,0.333333,0.754717,0.732601,0.047297,0.050419,0.018159,0.002758,0.002559,0.058183,0.038062,0.0,0.5
24860,3674727,7.792863,0.622154,0.281962,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.052158,0.052885,0.005567,0.002504,0.0,0.0
3398,9503002,6.925751,0.694079,0.244371,0.000000,0.000000,0.000000,0.006757,0.207200,0.005110,0.000000,0.000000,0.005320,0.000904,0.0,0.5
3161,6965817,6.591554,0.001123,0.030690,0.083333,0.094340,0.091575,0.006757,0.031588,0.050434,0.049724,0.050637,0.025712,0.007216,0.5,0.5
23527,12185175,6.155252,0.000051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.577499,0.0,0.0
169,3806667,6.103996,0.017815,0.013890,0.000000,0.000000,0.000000,0.263514,0.954846,0.114836,0.133112,0.137063,0.008514,0.054930,0.0,0.0


In [37]:
s.sort_values('score')[['uid', 'score']][:30]

,uid,score
23766,76791567,0.000239
18940,10330373,0.000439
19910,6434561,0.001097
18596,74617292,0.001165
19728,76146658,0.001248
23835,12210110,0.001452
19226,74630181,0.001452
32750,76795313,0.002204
28355,9508492,0.002374
33047,76792620,0.002410


In [38]:
s.sort_values('score', ascending=False)[['uid', 'score']].to_csv("power_seller_score_2.csv", index=False)

In [33]:
len(uid_list)

34208

In [56]:
s[s['uid'].isin(final_uid_list)].sort_values('score', ascending=False)[['uid', 'score']].to_csv("power_seller_score_3.csv", index=False)